In [2]:
print("Decision Tree Program")
print("Arshan Ahmad- 18BCS075")

Decision Tree Program
Arshan Ahmad- 18BCS075


In [8]:
import math
file=open('decision_tree.csv')
for line in file: 
  line=line.split(',')
  print(line)
#   for i in range(1,len(line)):
#     print(line[i])

['yes', 'no', 'no', 'yes', 'y \n']
['no', 'no', 'no', 'no', 'n\n']
['no', 'no', 'yes', 'no', 'n\n']
['yes', 'no', 'yes', 'no', 'y\n']
['no', 'no', 'sometimes', 'yes', 'n\n']
['no', 'no', 'no', 'yes', 'n\n']
['yes', 'yes', 'no', 'yes', 'y\n']
['no', 'yes', 'no', 'yes', 'n\n']
['yes', 'no', 'no', 'yes', 'y\n']
['yes', 'no', 'yes', 'no', 'n\n']
['no', 'no', 'sometimes', 'yes', 'n\n']
['no', 'no', 'sometimes', 'yes', 'n\n']
['yes', 'no', 'no', 'yes', 'y\n']
['no', 'no', 'yes', 'no', 'n\n']
['no', 'no', 'sometimes', 'yes', 'n\n']
['no', 'no', 'no', 'yes', 'n\n']
['no', 'no', 'no', 'yes', 'y\n']
['no', 'yes', 'no', 'yes', 'y']


In [11]:
def input_from_file(): 
    with open("decision_tree.csv") as f:
        data=f.read()
        rows=data.split('\n')
        print(rows)
        birth=[]
        fly=[]
        legs=[]
        water=[]
        classification=[]
        # for row in rows:
        #     birth.append(row.split(',')[1])
        #     fly.append(row.split(',')[2])
        #     legs.append(row.split(',')[4])
        #     water.append(row.split(',')[3])
        #     classification.append(row.split(',')[5])

In [12]:
file = open('decision_tree.csv')#change it
lines =file.readlines()
x=[]
p=[set() for i in range(len(lines[0].split(' ')))]
pd=[{} for i in range(len(lines[0].split(' ')))]
for line in lines:
  line=line.split(' ')
  y=[]
  k=0
  for i in range(1,len(line)): #change it 
    p[i].add(line[i])
print(p)

[set(), {'\n'}]


In [4]:
def calc_total_entropy(train_data, label, class_list):
    total_row = train_data.shape[0] #the total size of the dataset
    total_entr = 0
    
    for c in class_list: #for each class in the label
        total_class_count = train_data[train_data[label] == c].shape[0] #number of the class
        total_class_entr = - (total_class_count/total_row)*np.log2(total_class_count/total_row) #entropy of the class
        total_entr += total_class_entr #adding the class entropy to the total entropy of the dataset
    
    return total_entr

In [5]:
def calc_entropy(feature_value_data, label, class_list):
    class_count = feature_value_data.shape[0]
    entropy = 0
    
    for c in class_list:
        label_class_count = feature_value_data[feature_value_data[label] == c].shape[0] #row count of class c 
        entropy_class = 0
        if label_class_count != 0:
            probability_class = label_class_count/class_count #probability of the class
            entropy_class = - probability_class * np.log2(probability_class)  #entropy
        entropy += entropy_class
    return entropy

In [6]:
def calc_info_gain(feature_name, train_data, label, class_list):
    feature_value_list = train_data[feature_name].unique() #unqiue values of the feature
    total_row = train_data.shape[0]
    feature_info = 0.0
    
    for feature_value in feature_value_list:
        feature_value_data = train_data[train_data[feature_name] == feature_value] #filtering rows with that feature_value
        feature_value_count = feature_value_data.shape[0]
        feature_value_entropy = calc_entropy(feature_value_data, label, class_list) #calculcating entropy for the feature value
        feature_value_probability = feature_value_count/total_row
        feature_info += feature_value_probability * feature_value_entropy #calculating information of the feature value
        
    return calc_total_entropy(train_data, label, class_list) - feature_info #calculating information gain by subtracting


In [7]:
def find_most_informative_feature(train_data, label, class_list):
    feature_list = train_data.columns.drop(label) #finding the feature names in the dataset
                                            #N.B. label is not a feature, so dropping it
    max_info_gain = -1
    max_info_feature = None
    
    for feature in feature_list:  #for each feature in the dataset
        feature_info_gain = calc_info_gain(feature, train_data, label, class_list)
        if max_info_gain < feature_info_gain: #selecting feature name with highest information gain
            max_info_gain = feature_info_gain
            max_info_feature = feature
            
    return max_info_feature

In [8]:
def generate_sub_tree(feature_name, train_data, label, class_list):
    feature_value_count_dict = train_data[feature_name].value_counts(sort=False) #dictionary of the count of unqiue feature value
    tree = {} #sub tree or node
    
    for feature_value, count in feature_value_count_dict.iteritems():
        feature_value_data = train_data[train_data[feature_name] == feature_value] #dataset with only feature_name = feature_value
        
        assigned_to_node = False #flag for tracking feature_value is pure class or not
        for c in class_list: #for each class
            class_count = feature_value_data[feature_value_data[label] == c].shape[0] #count of class c

            if class_count == count: #count of feature_value = count of class (pure class)
                tree[feature_value] = c #adding node to the tree
                train_data = train_data[train_data[feature_name] != feature_value] #removing rows with feature_value
                assigned_to_node = True
        if not assigned_to_node: #not pure class
            tree[feature_value] = "?" #should extend the node, so the branch is marked with ?
            
    return tree, train_data

In [9]:
def make_tree(root, prev_feature_value, train_data, label, class_list):
    if train_data.shape[0] != 0: #if dataset becomes enpty after updating
        max_info_feature = find_most_informative_feature(train_data, label, class_list) #most informative feature
        tree, train_data = generate_sub_tree(max_info_feature, train_data, label, class_list) #getting tree node and updated dataset
        next_root = None
        
        if prev_feature_value != None: #add to intermediate node of the tree
            root[prev_feature_value] = dict()
            root[prev_feature_value][max_info_feature] = tree
            next_root = root[prev_feature_value][max_info_feature]
        else: #add to root of the tree
            root[max_info_feature] = tree
            next_root = root[max_info_feature]
        
        for node, branch in list(next_root.items()): #iterating the tree node
            if branch == "?": #if it is expandable
                feature_value_data = train_data[train_data[max_info_feature] == node] #using the updated dataset
                make_tree(next_root, node, feature_value_data, label, class_list) #recursive call with updated dataset


In [10]:
def id3(train_data_m, label):
    train_data = train_data_m.copy() #getting a copy of the dataset
    tree = {} #tree which will be updated
    class_list = train_data[label].unique() #getting unqiue classes of the label
    make_tree(tree, None, train_data_m, label, class_list) #start calling recursion
    return tree

In [11]:
# global tree
# x=10


In [12]:
def predict(tree, instance):
    print("abc")
    print(instance)
    if not isinstance(tree, dict): #if it is leaf node
        return tree #return the value
    else:
        root_node = next(iter(tree)) #getting first key/feature name of the dictionary
        feature_value = instance[root_node] #value of the feature
        if feature_value in tree[root_node]: #checking the feature value in current tree node
            return predict(tree[root_node][feature_value], instance) #goto next feature
        else:
            return None

In [13]:
# def evaluate(tree, test_data_m, label):
#     correct_preditct = 0
#     wrong_preditct = 0
#     for index, row in test_data_m.iterrows(): #for each row in the dataset
#         result = predict(tree, test_data_m.iloc[index]) #predict the row
#         if result == test_data_m[label].iloc[index]: #predicted value and expected value is same or not
#             correct_preditct += 1 #increase correct count
#         else:
#             wrong_preditct += 1 #increase incorrect count
#     accuracy = correct_preditct / (correct_preditct + wrong_preditct) #calculating accuracy
#     return accuracy

In [14]:
def evaluate(tree, test_data_m, label):
    correct_preditct = 0
    wrong_preditct = 0
    for index, row in test_data_m.iterrows(): #for each row in the dataset
        result = predict(tree, test_data_m.iloc[index]) #predict the row
        if result == test_data_m[label].iloc[index]: #predicted value and expected value is same or not
            correct_preditct += 1 #increase correct count
        else:
            wrong_preditct += 1 #increase incorrect count
    accuracy = correct_preditct / (correct_preditct + wrong_preditct) #calculating accuracy
    return accuracy

In [15]:
tree = id3(train_data_m, 'class')
accuracy = evaluate(tree, test_data_m, 'class') #evaluating the test datasetprint(tree)
print(accuracy)
# x =predict(tree,test_data_m.iloc[3])
# print(x)